# Notebook 73: 逆伝播の誕生 ― 勾配を逆流させる

## Backpropagation: Flowing Gradients Backward

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第4章として、計算グラフ上で **勾配を逆流させる** 逆伝播（Backpropagation）をスカラー版で完全に実装します。

### 学習目標

1. 各ノードの `backward()` メソッドを完全に実装する
2. **逆伝播の自動実行** を実現する
3. 損失関数を追加し、**勾配チェック** で正しさを検証する
4. $y = \sigma(Wx + b)$ の全パラメータの勾配を自動計算する

### 前提知識

- Notebook 70-72 の内容
- 特に Notebook 72 で構築した計算グラフのクラス設計

---

## 目次

1. [逆伝播の原理：復習](#1-逆伝播の原理復習)
2. [ノードクラスの再定義（backward付き）](#2-ノードクラスの再定義backward付き)
3. [自動逆伝播の実装](#3-自動逆伝播の実装)
4. [損失関数ノードの追加](#4-損失関数ノードの追加)
5. [勾配チェックによる検証](#5-勾配チェックによる検証)
6. [完全な順伝播・逆伝播の実行](#6-完全な順伝播逆伝播の実行)
7. [勾配の可視化](#7-勾配の可視化)
8. [演習問題](#8-演習問題)
9. [まとめと次のステップ](#9-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 逆伝播の原理：復習

### 1.1 順伝播と逆伝播の対比

| 方向 | 流れるもの | 目的 |
|------|-----------|------|
| **順伝播** | 値（データ） | 入力から出力を計算 |
| **逆伝播** | 勾配（誤差信号） | 各パラメータの影響度を計算 |

### 1.2 逆伝播のルール

各ノードは、**上流から受け取った勾配** と **局所的な勾配** を掛けて、**下流へ伝播** します。

$$
\frac{\partial L}{\partial \text{入力}} = \frac{\partial L}{\partial \text{出力}} \times \frac{\partial \text{出力}}{\partial \text{入力}}
$$

In [ ]:
# 順伝播と逆伝播の対比図
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 共通設定
node_positions = [(1, 1.5), (3, 1.5), (5, 1.5), (7, 1.5)]
node_labels = ['x', 'f', 'g', 'y']

# 左: 順伝播
ax = axes[0]
ax.set_xlim(0, 8)
ax.set_ylim(0, 3)
ax.axis('off')

for (nx, ny), label in zip(node_positions, node_labels):
    circle = Circle((nx, ny), 0.4, facecolor='lightgreen', edgecolor='darkgreen', linewidth=2)
    ax.add_patch(circle)
    ax.text(nx, ny, label, ha='center', va='center', fontsize=14, fontweight='bold')

for i in range(3):
    ax.annotate('', xy=(node_positions[i+1][0] - 0.5, 1.5),
                xytext=(node_positions[i][0] + 0.5, 1.5),
                arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.text(4, 2.5, '順伝播: 値が前へ流れる', ha='center', fontsize=12, color='darkgreen')
ax.text(4, 0.5, r'$x \rightarrow f(x) \rightarrow g(f(x)) \rightarrow y$', ha='center', fontsize=12)

# 右: 逆伝播
ax = axes[1]
ax.set_xlim(0, 8)
ax.set_ylim(0, 3)
ax.axis('off')

for (nx, ny), label in zip(node_positions, node_labels):
    circle = Circle((nx, ny), 0.4, facecolor='lightyellow', edgecolor='orange', linewidth=2)
    ax.add_patch(circle)
    ax.text(nx, ny, label, ha='center', va='center', fontsize=14, fontweight='bold')

for i in range(3):
    ax.annotate('', xy=(node_positions[i][0] + 0.5, 1.5),
                xytext=(node_positions[i+1][0] - 0.5, 1.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax.text(4, 2.5, '逆伝播: 勾配が後ろへ流れる', ha='center', fontsize=12, color='darkred')
ax.text(4, 0.5, r'$\frac{\partial L}{\partial x} \leftarrow \frac{\partial L}{\partial f} \leftarrow \frac{\partial L}{\partial g} \leftarrow \frac{\partial L}{\partial y}$', 
        ha='center', fontsize=12)

plt.suptitle('順伝播 vs 逆伝播', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## 2. ノードクラスの再定義（backward付き）

Notebook 72 のクラスを拡張し、`backward()` メソッドを完全に実装します。

In [ ]:
class Node:
    """
    計算グラフのノード基底クラス（backward対応版）
    """
    
    _id_counter = 0
    
    def __init__(self, name=None):
        self.value = None      # 順伝播の結果
        self.grad = None       # 逆伝播の勾配（∂L/∂self）
        self.inputs = []       # 入力ノード
        self.outputs = []      # 出力ノード
        
        self.id = Node._id_counter
        Node._id_counter += 1
        self.name = name if name else f"node_{self.id}"
    
    def forward(self):
        raise NotImplementedError
    
    def backward(self):
        raise NotImplementedError
    
    def zero_grad(self):
        """勾配をゼロにリセット"""
        if self.grad is not None:
            self.grad = np.zeros_like(self.grad)
    
    def __repr__(self):
        val_str = f"{self.value.item():.4f}" if self.value is not None and self.value.size == 1 else str(self.value)
        grad_str = f"{self.grad.item():.4f}" if self.grad is not None and np.array(self.grad).size == 1 else str(self.grad)
        return f"{self.__class__.__name__}('{self.name}', val={val_str}, grad={grad_str})"


class Variable(Node):
    """
    入力変数ノード（学習パラメータまたは入力データ）
    """
    
    def __init__(self, value, name=None, requires_grad=True):
        super().__init__(name)
        self.value = np.atleast_1d(np.array(value, dtype=np.float64))
        self.requires_grad = requires_grad
        self.grad = np.zeros_like(self.value) if requires_grad else None
    
    def forward(self):
        return self.value
    
    def backward(self):
        # 葉ノードは勾配を受け取るだけ（伝播先がない）
        pass
    
    def set_value(self, value):
        self.value = np.atleast_1d(np.array(value, dtype=np.float64))
        if self.requires_grad:
            self.grad = np.zeros_like(self.value)


class Operation(Node):
    """
    演算ノードの基底クラス
    """
    
    def __init__(self, *inputs, name=None):
        super().__init__(name)
        self.inputs = list(inputs)
        
        for inp in self.inputs:
            inp.outputs.append(self)
    
    def _get_input_values(self):
        return [inp.value for inp in self.inputs]


print("基底クラスを定義しました")

### 2.1 加算ノード（Add）の逆伝播

In [ ]:
class Add(Operation):
    """
    加算: z = x + y
    
    順伝播: z = x + y
    逆伝播: 
        ∂L/∂x = ∂L/∂z × ∂z/∂x = ∂L/∂z × 1 = ∂L/∂z
        ∂L/∂y = ∂L/∂z × ∂z/∂y = ∂L/∂z × 1 = ∂L/∂z
    
    → 勾配をそのまま両方の入力に分配
    """
    
    def __init__(self, x, y, name=None):
        super().__init__(x, y, name=name or 'add')
    
    def forward(self):
        x_val, y_val = self._get_input_values()
        self.value = x_val + y_val
        return self.value
    
    def backward(self):
        """勾配をそのまま入力に分配"""
        x, y = self.inputs
        
        # 入力の勾配に加算（分岐がある場合に対応）
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        
        x.grad = x.grad + self.grad * 1.0  # ∂z/∂x = 1
        y.grad = y.grad + self.grad * 1.0  # ∂z/∂y = 1


# 加算ノードの逆伝播を図解
print("【加算ノード z = x + y の逆伝播】")
print("")
print("      ∂L/∂x = ∂L/∂z × 1")
print("     ↗")
print("∂L/∂z")
print("     ↘")
print("      ∂L/∂y = ∂L/∂z × 1")
print("")
print("→ 上流からの勾配をそのまま両方の入力へ分配")

### 2.2 乗算ノード（Multiply）の逆伝播

In [ ]:
class Multiply(Operation):
    """
    乗算: z = x × y
    
    順伝播: z = x × y
    逆伝播:
        ∂L/∂x = ∂L/∂z × ∂z/∂x = ∂L/∂z × y
        ∂L/∂y = ∂L/∂z × ∂z/∂y = ∂L/∂z × x
    
    → 勾配に「相手の値」を掛けて分配
    """
    
    def __init__(self, x, y, name=None):
        super().__init__(x, y, name=name or 'mul')
    
    def forward(self):
        x_val, y_val = self._get_input_values()
        self.value = x_val * y_val
        return self.value
    
    def backward(self):
        """勾配に相手の値を掛けて分配"""
        x, y = self.inputs
        x_val, y_val = self._get_input_values()
        
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        
        x.grad = x.grad + self.grad * y_val  # ∂z/∂x = y
        y.grad = y.grad + self.grad * x_val  # ∂z/∂y = x


# 乗算ノードの逆伝播を図解
print("【乗算ノード z = x × y の逆伝播】")
print("")
print("      ∂L/∂x = ∂L/∂z × y  ← 相手の値(y)を掛ける")
print("     ↗")
print("∂L/∂z")
print("     ↘")
print("      ∂L/∂y = ∂L/∂z × x  ← 相手の値(x)を掛ける")
print("")
print("→ 順伝播で使った相手の値を『記憶』しておく必要がある")

### 2.3 シグモイドノード（Sigmoid）の逆伝播

In [ ]:
class Sigmoid(Operation):
    """
    シグモイド: y = σ(x) = 1 / (1 + exp(-x))
    
    順伝播: y = σ(x)
    逆伝播:
        ∂L/∂x = ∂L/∂y × ∂y/∂x = ∂L/∂y × σ(x)(1 - σ(x))
    
    → 順伝播の出力値を使って効率的に計算
    """
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'sigmoid')
    
    def forward(self):
        x_val = self.inputs[0].value
        # 数値安定性のための実装
        self.value = np.where(
            x_val >= 0,
            1 / (1 + np.exp(-x_val)),
            np.exp(x_val) / (1 + np.exp(x_val))
        )
        return self.value
    
    def backward(self):
        """シグモイドの微分: σ'(x) = σ(x)(1 - σ(x))"""
        x = self.inputs[0]
        
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        
        # σ'(x) = σ(x)(1 - σ(x)) を使う（self.valueはσ(x)）
        sigmoid_derivative = self.value * (1 - self.value)
        x.grad = x.grad + self.grad * sigmoid_derivative


# シグモイドの微分を図解
print("【シグモイドノード y = σ(x) の逆伝播】")
print("")
print("∂L/∂y  →  ∂L/∂x = ∂L/∂y × σ(x)(1 - σ(x))")
print("")
print("→ σ(x)(1-σ(x)) は最大でも 0.25")
print("→ 層が深くなると勾配が消失しやすい")

### 2.4 その他の演算ノード

In [ ]:
class ReLU(Operation):
    """ReLU: y = max(0, x)"""
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'relu')
        self._mask = None
    
    def forward(self):
        x_val = self.inputs[0].value
        self._mask = (x_val > 0).astype(float)
        self.value = np.maximum(0, x_val)
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * self._mask


class Square(Operation):
    """二乗: y = x²"""
    
    def __init__(self, x, name=None):
        super().__init__(x, name=name or 'square')
    
    def forward(self):
        x_val = self.inputs[0].value
        self.value = x_val ** 2
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        x_val = x.value
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * 2 * x_val  # ∂(x²)/∂x = 2x


class ScalarMultiply(Operation):
    """スカラー倍: y = c × x"""
    
    def __init__(self, x, scalar, name=None):
        super().__init__(x, name=name or f'scale({scalar})')
        self.scalar = scalar
    
    def forward(self):
        x_val = self.inputs[0].value
        self.value = self.scalar * x_val
        return self.value
    
    def backward(self):
        x = self.inputs[0]
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        x.grad = x.grad + self.grad * self.scalar  # ∂(cx)/∂x = c


class Subtract(Operation):
    """減算: z = x - y"""
    
    def __init__(self, x, y, name=None):
        super().__init__(x, y, name=name or 'sub')
    
    def forward(self):
        x_val, y_val = self._get_input_values()
        self.value = x_val - y_val
        return self.value
    
    def backward(self):
        x, y = self.inputs
        if x.grad is None:
            x.grad = np.zeros_like(x.value)
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        x.grad = x.grad + self.grad * 1.0   # ∂z/∂x = 1
        y.grad = y.grad + self.grad * (-1.0)  # ∂z/∂y = -1


print("演算ノードを定義しました")

---

## 3. 自動逆伝播の実装

### 3.1 トポロジカルソートの逆順

逆伝播は、順伝播の **逆順** で実行します。

In [ ]:
def topological_sort(output_node):
    """計算グラフをトポロジカルソート"""
    visited = set()
    order = []
    
    def dfs(node):
        if id(node) in visited:
            return
        visited.add(id(node))
        
        if hasattr(node, 'inputs'):
            for inp in node.inputs:
                dfs(inp)
        
        order.append(node)
    
    dfs(output_node)
    return order


def forward_pass(output_node, verbose=False):
    """順伝播を自動実行"""
    sorted_nodes = topological_sort(output_node)
    
    if verbose:
        print("【順伝播】")
    
    for node in sorted_nodes:
        result = node.forward()
        if verbose:
            val_str = f"{result.item():.6f}" if result.size == 1 else str(result)
            print(f"  {node.name}: {val_str}")
    
    return output_node.value


def backward_pass(output_node, verbose=False):
    """
    逆伝播を自動実行
    
    1. トポロジカルソートの逆順でノードを処理
    2. 出力ノードの勾配を1に初期化
    3. 各ノードの backward() を呼び出す
    """
    sorted_nodes = topological_sort(output_node)
    reversed_nodes = sorted_nodes[::-1]  # 逆順
    
    # 出力ノードの勾配を 1 に初期化（∂L/∂L = 1）
    output_node.grad = np.ones_like(output_node.value)
    
    if verbose:
        print("\n【逆伝播】")
        print(f"  {output_node.name}: grad = 1（出発点）")
    
    # 逆順で逆伝播を実行
    for node in reversed_nodes:
        if isinstance(node, Operation):
            node.backward()
            if verbose:
                for inp in node.inputs:
                    grad_str = f"{inp.grad.item():.6f}" if inp.grad.size == 1 else str(inp.grad)
                    print(f"  {inp.name}: grad = {grad_str}")


print("自動順伝播・逆伝播を定義しました")

### 3.2 簡単な例でテスト

In [ ]:
# テスト: y = (a + b) × c
print("="*60)
print("テスト: y = (a + b) × c")
print("="*60)

# グラフ構築
a = Variable(2.0, name='a')
b = Variable(3.0, name='b')
c = Variable(4.0, name='c')

t = Add(a, b, name='t=a+b')
y = Multiply(t, c, name='y=t×c')

# 順伝播
forward_pass(y, verbose=True)

# 逆伝播
backward_pass(y, verbose=True)

print("\n【勾配の解釈】")
print(f"  ∂y/∂a = {a.grad.item():.1f}: a を1増やすと y は {a.grad.item():.1f} 増える")
print(f"  ∂y/∂b = {b.grad.item():.1f}: b を1増やすと y は {b.grad.item():.1f} 増える")
print(f"  ∂y/∂c = {c.grad.item():.1f}: c を1増やすと y は {c.grad.item():.1f} 増える")

# 手計算での検証
# y = (a + b) × c = ac + bc
# ∂y/∂a = c = 4
# ∂y/∂b = c = 4
# ∂y/∂c = a + b = 5
print("\n【検算】")
print(f"  ∂y/∂a = c = 4 ✓")
print(f"  ∂y/∂b = c = 4 ✓")
print(f"  ∂y/∂c = a + b = 5 ✓")

---

## 4. 損失関数ノードの追加

### 4.1 平均二乗誤差（MSE）損失

In [ ]:
class MSELoss(Operation):
    """
    平均二乗誤差損失: L = (y - t)² / 2
    
    ここで t は教師信号（定数として扱う）
    
    順伝播: L = (y - t)² / 2
    逆伝播: ∂L/∂y = y - t
    """
    
    def __init__(self, y, target, name=None):
        """
        Args:
            y: 予測値ノード
            target: 教師信号（スカラーまたは配列）
        """
        super().__init__(y, name=name or 'mse_loss')
        self.target = np.atleast_1d(np.array(target, dtype=np.float64))
    
    def forward(self):
        y_val = self.inputs[0].value
        self.value = 0.5 * np.sum((y_val - self.target) ** 2)
        return self.value
    
    def backward(self):
        y = self.inputs[0]
        y_val = y.value
        
        if y.grad is None:
            y.grad = np.zeros_like(y.value)
        
        # ∂L/∂y = y - t
        y.grad = y.grad + self.grad * (y_val - self.target)


# MSE損失の微分を確認
print("【MSE損失 L = (y - t)² / 2 の逆伝播】")
print("")
print("∂L/∂y = y - t")
print("")
print("→ 予測が正解より大きい(y > t): 正の勾配 → y を減らす方向")
print("→ 予測が正解より小さい(y < t): 負の勾配 → y を増やす方向")

---

## 5. 勾配チェックによる検証

### 5.1 数値微分との比較

逆伝播で計算した勾配が正しいか、数値微分と比較して検証します。

In [ ]:
def numerical_gradient(param_node, loss_node, h=1e-5):
    """
    パラメータの数値勾配を計算
    
    Args:
        param_node: 勾配を計算したいパラメータノード
        loss_node: 損失関数ノード
        h: 微小変化量
    
    Returns:
        数値勾配
    """
    original_value = param_node.value.copy()
    grad = np.zeros_like(original_value)
    
    for i in range(original_value.size):
        # +h
        param_node.value = original_value.copy()
        param_node.value.flat[i] += h
        forward_pass(loss_node)
        loss_plus = loss_node.value.item()
        
        # -h
        param_node.value = original_value.copy()
        param_node.value.flat[i] -= h
        forward_pass(loss_node)
        loss_minus = loss_node.value.item()
        
        # 中心差分
        grad.flat[i] = (loss_plus - loss_minus) / (2 * h)
    
    # 元の値に戻す
    param_node.value = original_value
    forward_pass(loss_node)
    
    return grad


def gradient_check(params, loss_node, tol=1e-5):
    """
    勾配チェック: 解析的勾配と数値勾配を比較
    
    Args:
        params: パラメータノードのリスト
        loss_node: 損失関数ノード
        tol: 許容誤差
    
    Returns:
        全てのパラメータでチェックが通ったか
    """
    print("\n【勾配チェック】")
    print("="*70)
    print(f"{'パラメータ':>12} | {'解析的勾配':>15} | {'数値勾配':>15} | {'相対誤差':>12} | 結果")
    print("-"*70)
    
    all_passed = True
    
    for param in params:
        analytical = param.grad
        numerical = numerical_gradient(param, loss_node)
        
        # 相対誤差の計算
        diff = np.abs(analytical - numerical)
        denom = np.maximum(np.abs(analytical), np.abs(numerical)) + 1e-10
        relative_error = np.max(diff / denom)
        
        passed = relative_error < tol
        all_passed = all_passed and passed
        status = "✓ OK" if passed else "✗ NG"
        
        anal_str = f"{analytical.item():.8f}" if analytical.size == 1 else str(analytical)
        num_str = f"{numerical.item():.8f}" if numerical.size == 1 else str(numerical)
        
        print(f"{param.name:>12} | {anal_str:>15} | {num_str:>15} | {relative_error:>12.2e} | {status}")
    
    print("-"*70)
    if all_passed:
        print("全てのパラメータで勾配チェックに合格しました！")
    else:
        print("警告: 一部のパラメータで勾配が一致しません")
    
    return all_passed


print("勾配チェック関数を定義しました")

---

## 6. 完全な順伝播・逆伝播の実行

### 6.1 y = σ(Wx + b) の完全な学習グラフ

In [ ]:
def build_neuron_with_loss(x_val, W_val, b_val, target):
    """
    y = σ(Wx + b) + MSE損失の計算グラフを構築
    
    計算グラフ:
        x ─┬─ (×) ─┬─ (+) ─── (σ) ─── y ─── (MSE) ─── L
           │       │                        │
        W ─┘       b                        t(教師)
    """
    # 入力データ（勾配不要）
    x = Variable(x_val, name='x', requires_grad=False)
    
    # 学習パラメータ（勾配必要）
    W = Variable(W_val, name='W', requires_grad=True)
    b = Variable(b_val, name='b', requires_grad=True)
    
    # 順伝播グラフ
    z1 = Multiply(W, x, name='z1=Wx')
    z2 = Add(z1, b, name='z2=z1+b')
    y = Sigmoid(z2, name='y=σ(z2)')
    
    # 損失
    loss = MSELoss(y, target, name='L')
    
    return {
        'x': x, 'W': W, 'b': b,
        'z1': z1, 'z2': z2, 'y': y,
        'loss': loss,
        'target': target
    }


# グラフの構築
graph = build_neuron_with_loss(
    x_val=2.0,
    W_val=0.5,
    b_val=-0.3,
    target=1.0
)

print("【計算グラフ: y = σ(Wx + b) + MSE損失】\n")
print(f"入力: x = {graph['x'].value.item()}")
print(f"パラメータ: W = {graph['W'].value.item()}, b = {graph['b'].value.item()}")
print(f"教師信号: t = {graph['target']}")

In [ ]:
# 順伝播と逆伝播の実行
print("\n" + "="*60)
print("順伝播・逆伝播の完全実行")
print("="*60)

# 順伝播
forward_pass(graph['loss'], verbose=True)

# 逆伝播
backward_pass(graph['loss'], verbose=True)

# 結果のまとめ
print("\n【結果のまとめ】")
print(f"  予測値 y = {graph['y'].value.item():.6f}")
print(f"  正解値 t = {graph['target']}")
print(f"  損失 L = {graph['loss'].value.item():.6f}")
print(f"")
print(f"  ∂L/∂W = {graph['W'].grad.item():.6f}")
print(f"  ∂L/∂b = {graph['b'].grad.item():.6f}")

In [ ]:
# 勾配チェック
params = [graph['W'], graph['b']]
gradient_check(params, graph['loss'])

### 6.2 逆伝播の詳細追跡

In [ ]:
def detailed_backward_trace(graph):
    """
    逆伝播の各ステップを詳細に追跡
    """
    print("\n" + "="*70)
    print("逆伝播の詳細追跡: y = σ(Wx + b), L = (y - t)²/2")
    print("="*70)
    
    # 値の取得
    x = graph['x'].value.item()
    W = graph['W'].value.item()
    b = graph['b'].value.item()
    z1 = graph['z1'].value.item()
    z2 = graph['z2'].value.item()
    y = graph['y'].value.item()
    t = graph['target']
    L = graph['loss'].value.item()
    
    print(f"\n【順伝播の値】")
    print(f"  x = {x}")
    print(f"  W = {W}, b = {b}")
    print(f"  z1 = Wx = {z1}")
    print(f"  z2 = z1 + b = {z2}")
    print(f"  y = σ(z2) = {y:.6f}")
    print(f"  t = {t}")
    print(f"  L = (y-t)²/2 = {L:.6f}")
    
    print(f"\n【逆伝播のステップ】")
    
    # Step 1: dL/dL = 1
    dL_dL = 1
    print(f"\nStep 1: 出発点")
    print(f"  dL/dL = {dL_dL}")
    
    # Step 2: dL/dy = y - t
    dL_dy = y - t
    print(f"\nStep 2: MSE損失ノード")
    print(f"  dL/dy = y - t = {y:.6f} - {t} = {dL_dy:.6f}")
    
    # Step 3: dL/dz2 = dL/dy × dy/dz2 = dL/dy × σ(z2)(1-σ(z2))
    sigmoid_deriv = y * (1 - y)
    dL_dz2 = dL_dy * sigmoid_deriv
    print(f"\nStep 3: シグモイドノード")
    print(f"  dy/dz2 = σ(z2)(1-σ(z2)) = {y:.6f} × {1-y:.6f} = {sigmoid_deriv:.6f}")
    print(f"  dL/dz2 = dL/dy × dy/dz2 = {dL_dy:.6f} × {sigmoid_deriv:.6f} = {dL_dz2:.6f}")
    
    # Step 4: 加算ノード (z2 = z1 + b)
    dL_dz1 = dL_dz2 * 1  # dz2/dz1 = 1
    dL_db = dL_dz2 * 1   # dz2/db = 1
    print(f"\nStep 4: 加算ノード (z2 = z1 + b)")
    print(f"  dz2/dz1 = 1, dz2/db = 1")
    print(f"  dL/dz1 = dL/dz2 × 1 = {dL_dz1:.6f}")
    print(f"  dL/db = dL/dz2 × 1 = {dL_db:.6f}")
    
    # Step 5: 乗算ノード (z1 = W × x)
    dL_dW = dL_dz1 * x   # dz1/dW = x
    dL_dx = dL_dz1 * W   # dz1/dx = W
    print(f"\nStep 5: 乗算ノード (z1 = W × x)")
    print(f"  dz1/dW = x = {x}, dz1/dx = W = {W}")
    print(f"  dL/dW = dL/dz1 × x = {dL_dz1:.6f} × {x} = {dL_dW:.6f}")
    print(f"  dL/dx = dL/dz1 × W = {dL_dz1:.6f} × {W} = {dL_dx:.6f}")
    
    print(f"\n【最終結果】")
    print(f"  ∂L/∂W = {dL_dW:.6f}")
    print(f"  ∂L/∂b = {dL_db:.6f}")
    
    # 自動計算との比較
    print(f"\n【自動計算との比較】")
    print(f"  ∂L/∂W: 手計算 = {dL_dW:.6f}, 自動 = {graph['W'].grad.item():.6f}")
    print(f"  ∂L/∂b: 手計算 = {dL_db:.6f}, 自動 = {graph['b'].grad.item():.6f}")


# 詳細追跡の実行
detailed_backward_trace(graph)

---

## 7. 勾配の可視化

### 7.1 計算グラフ上での勾配フロー

In [ ]:
def visualize_gradient_flow(graph):
    """
    計算グラフ上で順伝播の値と逆伝播の勾配を可視化
    """
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # ノード位置
    positions = {
        'x': (1, 3),
        'W': (1, 1),
        'b': (5, 1),
        'z1': (3, 2),
        'z2': (6, 2),
        'y': (9, 2),
        'loss': (12, 2),
    }
    
    # ========== 上: 順伝播 ==========
    ax = axes[0]
    ax.set_xlim(0, 14)
    ax.set_ylim(0, 4)
    ax.axis('off')
    ax.set_title('順伝播: 値が前へ流れる', fontsize=12, color='darkgreen')
    
    for name, (nx, ny) in positions.items():
        node = graph.get(name)
        if node is None:
            continue
        
        # 色の決定
        if name in ['W', 'b']:
            color = 'lightyellow'
        elif name == 'loss':
            color = 'lightcoral'
        elif name == 'x':
            color = 'lightblue'
        else:
            color = 'lightgreen'
        
        circle = Circle((nx, ny), 0.5, facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)
        
        # ラベルと値
        if hasattr(node, 'value') and node.value is not None:
            val = node.value.item() if node.value.size == 1 else '...'
            ax.text(nx, ny + 0.1, name, ha='center', va='center', fontsize=10, fontweight='bold')
            ax.text(nx, ny - 0.25, f'{val:.3f}', ha='center', va='center', fontsize=8, color='blue')
        else:
            ax.text(nx, ny, name, ha='center', va='center', fontsize=10, fontweight='bold')
    
    # エッジ（順方向）
    edges = [
        ('x', 'z1'), ('W', 'z1'), ('z1', 'z2'), ('b', 'z2'),
        ('z2', 'y'), ('y', 'loss')
    ]
    for start, end in edges:
        sx, sy = positions[start]
        ex, ey = positions[end]
        ax.annotate('', xy=(ex - 0.55, ey), xytext=(sx + 0.55, sy),
                    arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # 演算ラベル
    ax.text(2, 2.7, '×', fontsize=14, ha='center')
    ax.text(4.5, 2.7, '+', fontsize=14, ha='center')
    ax.text(7.5, 2.7, 'σ', fontsize=14, ha='center')
    ax.text(10.5, 2.7, 'MSE', fontsize=10, ha='center')
    
    # ========== 下: 逆伝播 ==========
    ax = axes[1]
    ax.set_xlim(0, 14)
    ax.set_ylim(0, 4)
    ax.axis('off')
    ax.set_title('逆伝播: 勾配が後ろへ流れる', fontsize=12, color='darkred')
    
    for name, (nx, ny) in positions.items():
        node = graph.get(name)
        if node is None:
            continue
        
        # 色の決定（パラメータを強調）
        if name in ['W', 'b']:
            color = 'orange'
        elif name == 'loss':
            color = 'lightcoral'
        else:
            color = 'lightyellow'
        
        circle = Circle((nx, ny), 0.5, facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)
        
        # ラベルと勾配
        ax.text(nx, ny + 0.1, name, ha='center', va='center', fontsize=10, fontweight='bold')
        if hasattr(node, 'grad') and node.grad is not None:
            grad = node.grad.item() if np.array(node.grad).size == 1 else '...'
            ax.text(nx, ny - 0.25, f'∇={grad:.3f}', ha='center', va='center', fontsize=8, color='red')
    
    # エッジ（逆方向）
    for start, end in edges:
        sx, sy = positions[start]
        ex, ey = positions[end]
        ax.annotate('', xy=(sx + 0.55, sy), xytext=(ex - 0.55, ey),
                    arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    
    plt.tight_layout()
    plt.show()


# 可視化
visualize_gradient_flow(graph)

### 7.2 勾配の意味を解釈

In [ ]:
# 勾配の影響を確認
print("【勾配の解釈】\n")

W_grad = graph['W'].grad.item()
b_grad = graph['b'].grad.item()

print(f"現在の状態:")
print(f"  予測 y = {graph['y'].value.item():.4f}")
print(f"  正解 t = {graph['target']}")
print(f"  損失 L = {graph['loss'].value.item():.6f}")
print()

print(f"勾配:")
print(f"  ∂L/∂W = {W_grad:.6f}")
if W_grad > 0:
    print(f"    → W を減らすと損失が下がる（W は大きすぎる）")
else:
    print(f"    → W を増やすと損失が下がる（W は小さすぎる）")

print(f"  ∂L/∂b = {b_grad:.6f}")
if b_grad > 0:
    print(f"    → b を減らすと損失が下がる（b は大きすぎる）")
else:
    print(f"    → b を増やすと損失が下がる（b は小さすぎる）")

# 学習ステップのシミュレーション
print(f"\n【1ステップの学習をシミュレート（学習率 η = 0.5）】")
lr = 0.5
W_new = graph['W'].value.item() - lr * W_grad
b_new = graph['b'].value.item() - lr * b_grad
print(f"  W: {graph['W'].value.item():.4f} → {W_new:.4f}")
print(f"  b: {graph['b'].value.item():.4f} → {b_new:.4f}")

# 新しいパラメータでの予測
z_new = W_new * graph['x'].value.item() + b_new
y_new = 1 / (1 + np.exp(-z_new))
L_new = 0.5 * (y_new - graph['target']) ** 2
print(f"\n  新しい予測 y = {y_new:.4f}（元: {graph['y'].value.item():.4f}）")
print(f"  新しい損失 L = {L_new:.6f}（元: {graph['loss'].value.item():.6f}）")
print(f"  損失の変化: {L_new - graph['loss'].value.item():.6f}（負なら改善）")

---

## 8. 演習問題

### 演習 8.1: 2入力ニューロンの勾配計算

$y = \sigma(w_1 x_1 + w_2 x_2 + b)$ の計算グラフを構築し、全パラメータの勾配を計算してください。

In [ ]:
# 演習 8.1: 解答欄

def build_two_input_neuron(x1_val, x2_val, w1_val, w2_val, b_val, target):
    """
    y = σ(w1*x1 + w2*x2 + b) + MSE損失の計算グラフを構築
    
    TODO: 以下を実装
    1. 入力変数（x1, x2）と学習パラメータ（w1, w2, b）を定義
    2. 計算グラフを構築
    3. MSE損失を追加
    4. 辞書で返す
    """
    pass


# テストコード（実装後にコメントを外して実行）
# graph2 = build_two_input_neuron(1.0, 2.0, 0.5, -0.3, 0.1, 0.8)
# forward_pass(graph2['loss'], verbose=True)
# backward_pass(graph2['loss'], verbose=True)
# gradient_check([graph2['w1'], graph2['w2'], graph2['b']], graph2['loss'])

### 演習 8.2: ReLU活性化関数を使った勾配計算

シグモイドの代わりにReLUを使って同様の計算グラフを構築し、勾配を計算してください。

In [ ]:
# 演習 8.2: 解答欄

def build_relu_neuron(x_val, W_val, b_val, target):
    """
    y = ReLU(Wx + b) + MSE損失の計算グラフを構築
    
    TODO: 実装
    """
    pass


# テストコード

### 演習 8.3: 分岐を持つグラフ

$y = x^2 + 2x$ のように、$x$ が複数の経路に分岐するグラフを構築し、勾配が正しく加算されることを確認してください。

In [ ]:
# 演習 8.3: 解答欄

def build_branching_graph(x_val):
    """
    y = x² + 2x の計算グラフを構築
    
    注意: x は2つの経路（x² と 2x）に分岐する
    逆伝播では、両経路からの勾配が x に加算されるべき
    
    解析的な勾配: dy/dx = 2x + 2
    
    TODO: 実装
    """
    pass


# テストコード

---

## 9. まとめと次のステップ

### このノートブックで学んだこと

1. **逆伝播の原理**: 上流からの勾配 × 局所的な勾配 = 下流への勾配

2. **各ノードの backward()** 実装:
   - Add: 勾配をそのまま分配
   - Multiply: 勾配に相手の値を掛けて分配
   - Sigmoid: 勾配に σ(x)(1-σ(x)) を掛ける

3. **自動逆伝播**: トポロジカルソートの逆順で backward() を呼び出す

4. **勾配チェック**: 数値微分との比較で実装を検証

5. **勾配の意味**: パラメータをどの方向にどれだけ変化させれば損失が下がるか

### 次のノートブック（74: 行列で並列化）への橋渡し

このノートブックでは **スカラー** を扱いました。次のノートブックでは：

- **バッチ処理**: 複数のサンプルを同時に処理
- **行列微分**: ヤコビアンの概念
- **効率的な実装**: NumPyによるベクトル化

を学び、実用的なニューラルネットワークエンジンを構築します。

---

## 付録: 逆伝播の公式集

| ノード | 順伝播 | 逆伝播（∂L/∂入力） |
|--------|--------|--------------------|
| Add (z = x + y) | z = x + y | ∂L/∂x = ∂L/∂z, ∂L/∂y = ∂L/∂z |
| Multiply (z = xy) | z = xy | ∂L/∂x = y·∂L/∂z, ∂L/∂y = x·∂L/∂z |
| Sigmoid (y = σ(x)) | y = 1/(1+e⁻ˣ) | ∂L/∂x = y(1-y)·∂L/∂y |
| ReLU (y = max(0,x)) | y = max(0,x) | ∂L/∂x = 1_{x>0}·∂L/∂y |
| Square (y = x²) | y = x² | ∂L/∂x = 2x·∂L/∂y |
| MSE (L = (y-t)²/2) | L = (y-t)²/2 | ∂L/∂y = (y-t)·∂L/∂L |